In [10]:
import xmlrpclib
import time
import os.path
import errno

from collections import defaultdict
from pprint import pprint
from multiprocessing import Pool
from socket import error as socket_error

import cPickle

In [11]:
# Internal configurations
# Do not change this value unless you really know what you are doing.
MAX_REQUEST_SIMULTANEOUS = 16
CACHE_PATH = ".cache"
STATES_CACHE_FILE = "css_queries_cache.deepblue"
DEEPBLUE_URL = "http://deepblue.mpi-inf.mpg.de/xmlrpc"
#DEEPBLUE_URL = "http://localhost:31415"
USER_KEY = 'anonymous_key'

# User parameters (TODO: move to args)
GENOME = 'GRCh38'


In [12]:
def request_data(request_id):
    (status, info) = SERVER.info(request_id, USER_KEY)
    request_status = info[0]["state"]

    while request_status != "done" and request_status != "failed":
        print info
        time.sleep(10)
        (status, info) = SERVER.info(request_id, USER_KEY)
        request_status = info[0]["state"]

    print "Downloading data"
    (status, data) = SERVER.get_request_data(request_id, USER_KEY)
    if status != "okay":
        print data
        return None

    
    return data

In [13]:

def split_file(data):
    server_worker = xmlrpclib.Server(DEEPBLUE_URL, allow_none=True)

    [css_file, states] = data
    result = []

    while True:
        try:
            _, css_file_query_id = server_worker.select_experiments(
                css_file, None, None, None, USER_KEY)
            _, css_file_query_cache_id = server_worker.query_cache(
                css_file_query_id, True, USER_KEY)
            for state in states:
                _, css_file_filter_query_id = server_worker.filter_regions(
                    css_file_query_cache_id, "NAME", "==", state, "string", USER_KEY)
                result.append((state, css_file_filter_query_id))

            print css_file
            return css_file, result
        except socket_error as serr:
            print serr

In [14]:
def get_chromatin_stes(genome, user_key):

    status, csss_query_id = SERVER.select_regions(
        None, genome, "Chromatin State Segmentation",
        None, None, None, None, None, None, user_key)
    print status, csss_query_id

    status, csss_names_request_id = SERVER.distinct_column_values(
        csss_query_id, "NAME", user_key)
    print csss_names_request_id
    distinct_values = request_data(csss_names_request_id)
    print distinct_values['distinct']

    return distinct_values['distinct']

In [15]:
def build_chromatin_state_files(server, genome, user_key):
    cache_file = os.path.join(CACHE_PATH, STATES_CACHE_FILE)

    if os.path.exists(cache_file):
        _file = open(cache_file, "r")
        queries = cPickle.load(_file)
        return queries
    else:
        _, css_files_list = server.list_experiments(
            genome, "peaks", "Chromatin State Segmentation", None, None, None, None, user_key)
        _, css_files_names = server.extract_names(css_files_list)
        states = get_chromatin_stes(genome, user_key)

        css_files = []
        for css_file in css_files_names:
            css_files.append([css_file, states])

        pool = Pool(MAX_REQUEST_SIMULTANEOUS)
        queries = pool.map(split_file, css_files)
        pprint(queries)
        datasets = defaultdict(list)

        for query in queries:
            for state in query[1]:
                datasets[query[0]].append([state[1], state[0]])

        datasets = dict(datasets)

        print datasets

        pool.close()
        pool.join()

        try:
            os.makedirs(CACHE_PATH)
        except OSError as exc:  # Python >2.5
            if exc.errno == errno.EEXIST and os.path.isdir(CACHE_PATH):
                pass
            else:
                raise

        _file = open(cache_file, "w+")
        # print queries
        cPickle.dump(datasets, _file)
        return datasets

In [16]:
SERVER = xmlrpclib.Server(DEEPBLUE_URL, allow_none=True)
print SERVER.echo(USER_KEY)

# Query data
# Select the hypo methylated regions where the AVG methyl level is lower
# than 0.0025
_, QUERY_ID = SERVER.select_experiments(
    "S00VEQA1.hypo_meth.bs_call.GRCh38.20150707.bed", None, None, None, USER_KEY)
_, QUERY_ID = SERVER.filter_regions(
    QUERY_ID, "AVG_METHYL_LEVEL", "<", "0.0025", "number", USER_KEY)
print QUERY_ID

# Universe will be tiling regions of 1000bp
_, UNIVERSE_QUERY_ID = SERVER.tiling_regions(1000, "grch38", None, USER_KEY)

# Datasets will be the chromatin segmentation files divided by segments
DATASETS = build_chromatin_state_files(SERVER, GENOME, USER_KEY)
print "total of " + str(len(DATASETS)) + " datasets"



['okay', 'DeepBlue (1.17.7) says hi to anonymous']
q978455
total of 173 datasets


In [17]:
__, ENRICH_REQUEST = SERVER.enrich_regions_overlap(QUERY_ID, UNIVERSE_QUERY_ID, DATASETS, "grch38", USER_KEY)
print "Processing enrich_region_overlap: ", ENRICH_REQUEST

ENRICHMENT_RESULT = request_data(ENRICH_REQUEST)

Processing enrich_region_overlap:  r822256


In [18]:
state_rank = [(k["description"], k["mean_rank"]) for k in ENRICHMENT_RESULT["enrichment"]["results"]]
pprint(state_rank)

[('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 30.6667),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 33.3333),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 37.6667),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 40.3333),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 40.6667),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 42.6667),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 53.0),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 54.0),
 ('11_Active_TSS_High_Signal_H3K4me3_H3K4me1', 55.0),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 55.0),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 55.0),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 55.6667),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 56.0),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 56.3333),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 56.6667),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 56.6667),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 56.6667),
 ('12_Active_TSS_High_Signal_H3K4me3_H3K27Ac', 57

 ('3_Low_signal', 719.3333),
 ('3_Low_signal', 721.0),
 ('3_Low_signal', 722.3333),
 ('3_Low_signal', 723.6667),
 ('3_Low_signal', 727.6667),
 ('3_Low_signal', 730.6667),
 ('3_Low_signal', 735.3333),
 ('3_Low_signal', 737.0),
 ('3_Low_signal', 738.0),
 ('3_Low_signal', 738.6667),
 ('3_Low_signal', 741.0),
 ('3_Low_signal', 741.6667),
 ('3_Low_signal', 744.0),
 ('3_Low_signal', 744.6667),
 ('3_Low_signal', 745.3333),
 ('3_Low_signal', 747.6667),
 ('3_Low_signal', 749.3333),
 ('3_Low_signal', 750.0),
 ('3_Low_signal', 751.6667),
 ('3_Low_signal', 753.0),
 ('3_Low_signal', 754.3333),
 ('3_Low_signal', 758.0),
 ('3_Low_signal', 758.6667),
 ('3_Low_signal', 759.6667),
 ('3_Low_signal', 764.6667),
 ('3_Low_signal', 766.6667),
 ('3_Low_signal', 768.0),
 ('3_Low_signal', 768.0),
 ('3_Low_signal', 768.3333),
 ('3_Low_signal', 768.6667),
 ('3_Low_signal', 769.6667),
 ('3_Low_signal', 770.0),
 ('3_Low_signal', 770.0),
 ('3_Low_signal', 773.3333),
 ('3_Low_signal', 773.6667),
 ('3_Low_signal', 774

 ('4_Heterochromatin_High', 1756.0),
 ('7_Genic_Enhancer_High', 1756.6666),
 ('7_Genic_Enhancer_High', 1757.0),
 ('4_Heterochromatin_High', 1758.3334),
 ('7_Genic_Enhancer_High', 1759.0),
 ('4_Heterochromatin_High', 1759.3334),
 ('4_Heterochromatin_High', 1760.6666),
 ('4_Heterochromatin_High', 1763.0),
 ('4_Heterochromatin_High', 1764.0),
 ('4_Heterochromatin_High', 1764.0),
 ('7_Genic_Enhancer_High', 1764.3334),
 ('4_Heterochromatin_High', 1765.3334),
 ('4_Heterochromatin_High', 1765.3334),
 ('4_Heterochromatin_High', 1767.0),
 ('4_Heterochromatin_High', 1768.0),
 ('5_Transcription_High', 1770.3334),
 ('4_Heterochromatin_High', 1770.6666),
 ('4_Heterochromatin_High', 1771.6666),
 ('4_Heterochromatin_High', 1774.3334),
 ('4_Heterochromatin_High', 1775.0),
 ('4_Heterochromatin_High', 1777.0),
 ('4_Heterochromatin_High', 1777.3334),
 ('5_Transcription_High', 1777.6666),
 ('4_Heterochromatin_High', 1778.0),
 ('4_Heterochromatin_High', 1780.6666),
 ('4_Heterochromatin_High', 1782.0),
 ('5